# Data prep

For the fraud detection [tutorial](./fraud_detection.ipynb), the first step is to download the [credict card fraud dataset](https://www.kaggle.com/mlg-ulb/creditcardfraud) and the coordinate locations of the [world cities dataset](https://simplemaps.com/data/world-cities). We are going to save it in a SQLite database.



In [1]:
import sys
import os
import pandas as pd
import sqlite3
from utils import DATABASE_FILE, TABLE_FRAUD, TABLE_LOCATIONS, save_to_sqlite, connect_to_database

%load_ext autoreload
%autoreload 2

#### Create fraud table and insert the values

Download the [credict card fraud dataset](https://www.kaggle.com/mlg-ulb/creditcardfraud) either manually or via the [Kaggle CLI](https://github.com/Kaggle/kaggle-api). With the CLI, you can use the command `kaggle datasets download -d mlg-ulb/creditcardfraud`. 

In [2]:
dataset_file = "creditcard.csv"
df = pd.read_csv(dataset_file)
print(df.shape)
df.head()

(284807, 31)


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [3]:
save_to_sqlite(df, DATABASE_FILE, TABLE_FRAUD, if_exists='replace', index=False)

In [4]:
# Make sure it's correct
conn = connect_to_database(DATABASE_FILE)
cur = conn.cursor()
cur.execute("SELECT * FROM " + TABLE_FRAUD)
print(cur.fetchone())
cur.execute("SELECT COUNT(*) FROM " + TABLE_FRAUD)
print(cur.fetchone())
conn.close()

(0.0, -1.3598071336738, -0.0727811733098497, 2.53634673796914, 1.37815522427443, -0.338320769942518, 0.462387777762292, 0.239598554061257, 0.0986979012610507, 0.363786969611213, 0.0907941719789316, -0.551599533260813, -0.617800855762348, -0.991389847235408, -0.311169353699879, 1.46817697209427, -0.470400525259478, 0.207971241929242, 0.0257905801985591, 0.403992960255733, 0.251412098239705, -0.018306777944153, 0.277837575558899, -0.110473910188767, 0.0669280749146731, 0.128539358273528, -0.189114843888824, 0.133558376740387, -0.0210530534538215, 149.62, 0)
(284807,)


#### Create locations table and insert the values

In [5]:
!wget https://simplemaps.com/static/data/world-cities/basic/simplemaps_worldcities_basicv1.75.zip -O simplemaps_worldcities_basic.zip


--2023-01-28 08:53:12--  https://simplemaps.com/static/data/world-cities/basic/simplemaps_worldcities_basicv1.75.zip
Resolving simplemaps.com (simplemaps.com)... 104.26.13.95, 104.26.12.95, 172.67.71.113, ...
Connecting to simplemaps.com (simplemaps.com)|104.26.13.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4390984 (4.2M) [application/zip]
Saving to: ‘simplemaps_worldcities_basic.zip’

simplemaps_worldcit 100%[===================>]   4.19M  12.5MB/s    in 0.3s    

2023-01-28 08:53:13 (12.5 MB/s) - ‘simplemaps_worldcities_basic.zip’ saved [4390984/4390984]



In [6]:
!unzip -o simplemaps_worldcities_basic.zip

Archive:  simplemaps_worldcities_basic.zip
  inflating: license.txt             
  inflating: worldcities.csv         
  inflating: worldcities.xlsx        


In [7]:
dataset_file = "worldcities.csv"
df = pd.read_csv(dataset_file, usecols=["city_ascii", "country", "lat", "lng"])
df = df.rename(columns={"city_ascii": "city", "lat": "latitude", "lng": "longitude"})
print(df.shape)
df.head()

(42905, 4)


,city,latitude,longitude,country
0,Tokyo,35.6839,139.7744,Japan
1,Jakarta,-6.2146,106.8451,Indonesia
2,Delhi,28.6667,77.2167,India
3,Manila,14.6000,120.9833,Philippines
4,Sao Paulo,-23.5504,-46.6339,Brazil


In [8]:
save_to_sqlite(df, DATABASE_FILE, TABLE_LOCATIONS, if_exists='replace', index=False)

In [9]:
# Make sure it's correct
conn = connect_to_database(DATABASE_FILE)
cur = conn.cursor()
cur.execute("SELECT * FROM " + TABLE_LOCATIONS)
print(cur.fetchone())
cur.execute("SELECT COUNT(*) FROM " + TABLE_LOCATIONS)
print(cur.fetchone())
conn.close()

('Tokyo', 35.6839, 139.7744, 'Japan')
(42905,)
